In [ ]:
import yaml
import pdfkit
from pathlib import Path
from openssa.core.agent.agent import Agent
from openssa.core.resource.file import FileResource
from IPython.display import display, Markdown
from semikong_lm import SemiKongLM
from openssa.core.programming.hierarchical.planner import HTPlanner
from openssa.core.program_space import ProgramSpace

Answering questions about networking chips with OpenSSA.


In [ ]:
DATA_DIR = "./data"
with open("reference.yml") as f:
    reference = yaml.safe_load(f)
    QUESTIONS = reference["questions"]
    DOCS = reference["docs"]

Import docs. Get the ones that failed by hand. You could also manually get the docs.


In [ ]:
# Path(DATA_DIR).mkdir(exist_ok=True)
# for i, link in enumerate(DOCS):
#     try:
#         pdfkit.from_url(link, f"{DATA_DIR}/doc_{i}.pdf")
#     except Exception as e:  # noqa: PERF203
#         print(f"Failed to download {link}: {e}")

Initialise SemiKong LM in the programmer and executor.


Create agent with SemiKong base model for both programmer and executor.


In [ ]:
agent = Agent(
    resources={FileResource(path=DATA_DIR)},
    programmer=HTPlanner(lm=SemiKongLM.from_defaults()),
    program_space=ProgramSpace(lm=SemiKongLM.from_defaults()),
)

Ask it a question. (The first run it needs to build the index.)


In [ ]:
question = QUESTIONS[5]
print(question)

In [ ]:
answer = agent.solve(question)
display(Markdown(answer))

Compare its performance to generic LLM.

You may notice generic LLMs will:

1. answer questions in a more general fashion,
2. list off multiple possible answers, or
3. not synthesize separate pieces of knowledge.


In [ ]:
import os
from openai import OpenAI

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

chat_completion = client.chat.completions.create(
    messages=[{"role": "user", "content": question}],
    model="gpt-4o",
)

display(Markdown(chat_completion.choices[0].message.content))